# Librerias

In [1]:
import pandas as pd
import numpy as np
import unicodedata
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix
)
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from category_encoders import TargetEncoder

# Datos

In [7]:
df=pd.read_csv(r'C:\Users\Usuario\Documents\Trabajo Ingenieria Modelos\Bases de Datos\train.csv', encoding="utf-8")
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# Preprocesamiento General

In [ ]:
if df['F_TIENEINTERNET'].equals(df['F_TIENEINTERNET.1']):
    df = df.drop(columns=['F_TIENEINTERNET.1'])
    print("Se elimino la variable F_TIENEINTERNET.1")

# Moda global para variables categóricas simples
variables_imputar = ['F_ESTRATOVIVIENDA', 'F_EDUCACIONMADRE', 'F_EDUCACIONPADRE']
for col in variables_imputar:
    print(f"\n📊 Variable: {col}")
    nulos_antes = df[col].isna().sum()
    print(f" - Valores nulos antes de imputar: {nulos_antes}")
    moda_valor = df[col].mode()[0]
    print(f" - Valor usado para imputar (moda): {moda_valor}")
    df.loc[:, col] = df[col].fillna(moda_valor)
    nulos_despues = df[col].isna().sum()
    print(f" - Valores nulos después de imputar: {nulos_despues}")
print("\n✅ Imputación completada correctamente.")

# Imputación por grupos para variables categóricas
imputaciones_grupales = {
    'F_TIENEAUTOMOVIL': 'F_ESTRATOVIVIENDA',
    'F_TIENELAVADORA': 'F_ESTRATOVIVIENDA',
    'F_TIENEINTERNET': 'F_ESTRATOVIVIENDA',
    'F_TIENECOMPUTADOR': 'F_TIENEINTERNET',
    'E_PAGOMATRICULAPROPIO': 'F_ESTRATOVIVIENDA',
    'E_HORASSEMANATRABAJA': 'F_ESTRATOVIVIENDA',
    'E_VALORMATRICULAUNIVERSIDAD': 'F_ESTRATOVIVIENDA'
}

for var, grupo in imputaciones_grupales.items():
    print(f"\n📊 Variable: {var} (agrupada por {grupo})")
    nulos_antes = df[var].isna().sum()
    print(f" - Valores nulos antes de imputar: {nulos_antes}")
    
    # Imputación por moda dentro del grupo
    df[var] = df.groupby(grupo)[var].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'No')
    )
    
    nulos_despues = df[var].isna().sum()
    print(f" - Valores nulos después de imputar: {nulos_despues}")

print("\n✅ Imputación por grupos completada correctamente.")

# Convertir PERIODO_ACADEMICO a categórica
df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype(str)
df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype("category")

def quitar_tildes(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize("NFD", texto)
        texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")
        return texto.upper()  
    return texto

df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].apply(quitar_tildes)
print("Homogeneización Realizada")

Se elimino la variable F_TIENEINTERNET.1


In [19]:
import pandas as pd
import unicodedata

def preprocesar_datos(df):
    df = df.copy()

    # -------------------------------------
    # 1. Eliminar duplicado F_TIENEINTERNET.1
    # -------------------------------------
    if "F_TIENEINTERNET.1" in df.columns:
        if df["F_TIENEINTERNET"].equals(df["F_TIENEINTERNET.1"]):
            df = df.drop(columns=["F_TIENEINTERNET.1"])
            print("🧹 Se eliminó la variable duplicada: F_TIENEINTERNET.1")

    # -------------------------------------
    # 2. Imputación de moda global
    # -------------------------------------
    variables_imputar = [
        "F_ESTRATOVIVIENDA",
        "F_EDUCACIONMADRE",
        "F_EDUCACIONPADRE"
    ]

    for col in variables_imputar:
        if col in df.columns:
            moda = df[col].mode()[0]
            df[col] = df[col].fillna(moda)

    print("📌 Imputación global completada")

    # -------------------------------------
    # 3. Imputación por grupos (estrato, internet)
    # -------------------------------------
    imputaciones_grupales = {
        'F_TIENEAUTOMOVIL': 'F_ESTRATOVIVIENDA',
        'F_TIENELAVADORA': 'F_ESTRATOVIVIENDA',
        'F_TIENEINTERNET': 'F_ESTRATOVIVIENDA',
        'F_TIENECOMPUTADOR': 'F_TIENEINTERNET',
        'E_PAGOMATRICULAPROPIO': 'F_ESTRATOVIVIENDA',
        'E_HORASSEMANATRABAJA': 'F_ESTRATOVIVIENDA',
        'E_VALORMATRICULAUNIVERSIDAD': 'F_ESTRATOVIVIENDA'
    }

    for col, grupo in imputaciones_grupales.items():
        if col in df.columns and grupo in df.columns:

            df[col] = df.groupby(grupo)[col].transform(
                lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "No")
            )

    print("📌 Imputación por grupos completada")

    # -------------------------------------
    # 4. Convertir PERIODO_ACADEMICO a categoría
    # -------------------------------------
    if "PERIODO_ACADEMICO" in df.columns:
        df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype(str).astype("category")

    # -------------------------------------
    # 5. Función para quitar tildes
    # -------------------------------------
    def quitar_tildes(texto):
        if isinstance(texto, str):
            texto = unicodedata.normalize("NFD", texto)
            texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")
            return texto.upper()
        return texto

    # Aplicar a PRGM_ACADEMICO si existe
    if "E_PRGM_ACADEMICO" in df.columns:
        df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].apply(quitar_tildes)

    print("✨ Homogeneización de textos realizada")
    print("✅ Preprocesamiento completado")

    return df


# LightGBM

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------
# 1. IDENTIFICAR VARIABLES
# -----------------------------
cat_cols = [
    'F_ESTRATOVIVIENDA', 'F_EDUCACIONMADRE', 'F_EDUCACIONPADRE',
    'F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
    'F_TIENECOMPUTADOR', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO',
    'E_HORASSEMANATRABAJA',
    'E_VALORMATRICULAUNIVERSIDAD'
]

num_cols = [
    'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'
]

X = df[cat_cols + num_cols]
y = df["RENDIMIENTO_GLOBAL"]

# -----------------------------
# 2. PREPROCESAMIENTO
# -----------------------------
preprocessor = ColumnTransformer([
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("enc", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
    ]), cat_cols),
    
    ("num", SimpleImputer(strategy="median"), num_cols)
])

# -----------------------------
# 3. MODELO
# -----------------------------
pipe = Pipeline([
    ("prep", preprocessor),
    ("model", HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_depth=6,
        max_iter=300,
        random_state=42
    ))
])

# -----------------------------
# 4. ENTRENAMIENTO
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

# -----------------------------
# 5. RESULTADOS
# -----------------------------
print("\n=== HIST GRADIENT BOOSTING ===")
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))



=== HIST GRADIENT BOOSTING ===
              precision    recall  f1-score   support

        alto       0.48      0.60      0.54     52686
        bajo       0.40      0.53      0.46     51896
  medio-alto       0.30      0.20      0.24     51486
  medio-bajo       0.31      0.25      0.28     51682

    accuracy                           0.39    207750
   macro avg       0.37      0.39      0.38    207750
weighted avg       0.38      0.39      0.38    207750

[[31664  7076  8035  5911]
 [ 7023 27269  6649 10955]
 [16660 13611 10153 11062]
 [10185 19880  8864 12753]]


## EJECUCIÓN MODELO EN TEST

In [20]:
df_test=pd.read_csv(r'C:\Users\Usuario\Documents\Trabajo Ingenieria Modelos\Bases de Datos\test.csv', encoding="utf-8")
df_test.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [21]:
df_test_prep = preprocesar_datos(df_test)
pred_test = pipe.predict(df_test_prep)

df_resultado = pd.DataFrame({
    "ID": df_test["ID"],            
    "RENDIMIENTO_GLOBAL_PREDICHO": pred_test
})

# ============================================================
# 5. EXPORTAR RESULTADO
# ============================================================

df_resultado.to_csv("predicciones_finales.csv", index=False)
df_resultado.head()

🧹 Se eliminó la variable duplicada: F_TIENEINTERNET.1
📌 Imputación global completada
📌 Imputación por grupos completada
✨ Homogeneización de textos realizada
✅ Preprocesamiento completado


,ID,RENDIMIENTO_GLOBAL_PREDICHO
0,550236,alto
1,98545,medio-bajo
2,499179,medio-alto
3,782980,bajo
4,785185,medio-bajo
